In [1]:
# conda install -c conda-forge cassandra-driver
import time
start_time = time.time()
import pandas as pd
import numpy as np
import datetime as dt
import sqlite3
from sqlite3 import Error
from datetime import datetime
import requests
import json
import ast
import pyodbc
from cassandra.cluster import Cluster
import warnings
warnings.filterwarnings("ignore")
yesterday = dt.date.today() - dt.timedelta(1)
t = (yesterday - dt.date(1970, 1, 1)).total_seconds()
FMT = '%Y-%m-%d %H:%M:%S'

In [2]:
def preprocess(dat):
    data=dat[['imei', 'devicetime', 'devicedata']]
    data_json=data.to_json(orient='records')
    dicti = ast.literal_eval(data_json)
    return(dicti)

# Send whole data with this function

In [39]:
def send_data(data):
    global tasks
    dicti=preprocess(data)
    data={'data':dicti}
    post_call = requests.post("http://127.0.0.1:5000/accept_data/",json=data)
    print( post_call, "POST call")
    print( post_call.text, "TEXT")
    print( post_call.content, "CONTENT")
    print(post_call.status_code, "STATUS CODE")
    return("Done")

In [4]:
def execute_query(imei=[]): #Give input imei's in a list
    query = "select *from devicedata where imei In "+ str(imei) +" and devicetime>=" + str(int(t)) +" and devicetime<=" + str(int(t+86399))+"  ALLOW FILTERING;"
    #print(str(imei))
    #print('Connecting to server...')
    cluster = Cluster(['45.114.143.13'])
    session = cluster.connect('assettl_tma')
    data = pd.DataFrame(list(session.execute(query)))
    time = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
    data.to_csv(r"C:\Users\INFO-DSK-04\Desktop\Asset_Data\Downloaded Data\downloded_on "+str(time)+".csv")
    sql_data=data[["imei","devicetime","devicedata"]]
    save_sql_data(sql_data)
    session.shutdown()
    return(data)

In [5]:
def save_sql_data(data):
    sql_conn = pyodbc.connect(driver='SQL Server',
                         host='INFO-DSK-04',
                         database='Asset_Data',
                         trusted_connection='yes')
    cursor = sql_conn.cursor()
    cursor.execute('''TRUNCATE TABLE Previous_day_data''')
    for index, row in data.iterrows():
        cursor.execute('''INSERT INTO [dbo].[Previous_day_data]([imei],[devicetime],[devicedata]) values(?,?,?)''', row['imei'],row['devicetime'],row['devicedata'])
    sql_conn.commit()
    sql_conn.close()
    return("SQL Data Saved")

In [6]:
def run_whole_process(imei):
    start_time = time.time()
    print("Starting Process to get data")
    data=execute_query(imei)
    print("Starting Process to Process Data")
    send_data(data)
    print("Time taken","--- %s seconds ---" % (time.time() - start_time))
    return("Done")

In [7]:
imei=('911615400067990','861359036048302','861359036046330','869867030115077','869867030123980','869867030133294','869867030717930','869867030740296','869867030740858','869867030759999','869867030760005','869867030129649','869867030138533','869867030716288','869867030723334','869867030737730','869867030737995')

In [14]:
run_whole_process(imei)

Starting Process to get data


NoHostAvailable: ('Unable to connect to any servers', {'45.114.143.13:9042': ConnectionRefusedError(10061, "Tried connecting to [('45.114.143.13', 9042)]. Last error: No connection could be made because the target machine actively refused it")})

In [11]:
data = pd.read_csv(r'C:\Users\INFO-DSK-04\Downloads\cassandra_data_20_11-2019.csv')

In [42]:
send_data(data)

<Response [200]> POST call
OK TEXT
b'OK' CONTENT
200 STATUS CODE


'Done'